# NER

In [121]:
# Import libraries
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import re
import datasets
from datasets import Dataset, DatasetDict
# from datasets import load_dataset
import evaluate

import transformers
from transformers import BertTokenizerFast, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline, TFBertForSequenceClassification, create_optimizer


from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization, Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

In [87]:
df = pd.read_csv('../data/labelled_reviews.csv')

In [91]:
df = df.rename(columns = {'Convenient':'convenient',
                     'Speed':'speed',
                     'Informative':'informative'})

In [92]:
df = df[['review', 'convenient', 'speed', 'informative']]

In [95]:
# Define a regex pattern to match URLs
url_pattern = re.compile(r'https?://\S+')

def remove_urls(text):
    if text == '':
        return 
    # Remove URLs
    text = url_pattern.sub('', text)
    # Remove commas
    text = text.replace(',', '')
    return text

In [100]:
df = df.fillna('')

In [101]:
# Apply the function to the 'text' column and create a new column 'clean_text'
df['review'] = df['review'].apply(remove_urls)
df['convenient'] = df['convenient'].apply(remove_urls)
df['speed'] = df['speed'].apply(remove_urls)
df['informative'] = df['informative'].apply(remove_urls)

In [102]:
# Replace all the punctuations and white space characters
df = df.replace(to_replace=r'[^\w\s]', value='', regex=True)
# Change word to lowercase
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# Remove the digits
df = df.replace(to_replace=r'\d', value='', regex=True)

C:\Users\Yang LiTing\AppData\Local\Temp\ipykernel_154964\3864769724.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [105]:
def map_tags(lst, type:str):
  if not isinstance(lst, str) or len(lst.strip()) == 0:
        return {}
  # if len(lst) == 0:
  #   return {}
  dct = {}
  tokens = lst.split(" ")
  if type == 'convenient':
    inbet = 1
  elif type == 'speed':
    inbet = 2
  elif type == 'informative':
    inbet = 3
  for i in range(len(tokens)):
    if tokens[i] == '':
      continue
    dct[tokens[i]] = inbet
  print(dct)
  return dct

In [106]:
df['convenient'] = df['convenient'].apply(lambda x: map_tags(x, 'convenient'))
df['speed'] = df['speed'].apply(lambda x: map_tags(x, 'speed'))
df['informative'] = df['informative'].apply(lambda x: map_tags(x, 'informative'))

{'no': 1, 'difficulties': 1, 'faced': 1, 'net': 1, 'banking': 1, 'mobile': 1, 'service': 1}
{'smooth': 1, 'convenient': 1}
{'available': 1, 'in': 1, 'most': 1, 'of': 1, 'the': 1, 'areas': 1, 'very': 1, 'convenience': 1, 'for': 1, 'deposits': 1, 'and': 1, 'update': 1, 'passbook': 1}
{'easily': 1, 'transact': 1, 'withdraw': 1, 'money': 1, 'from': 1, 'anywhere': 1, 'atm': 1, 'and': 1, 'their': 1, 'branches': 1, 'are': 1, 'near': 1, 'to': 1, 'my': 1, 'residence': 1}
{'located': 1, 'nearby': 1, 'very': 1, 'easy': 1, 'convenience': 1, 'for': 1, 'money': 1, 'deposits': 1}
{'online': 1, 'zero': 1, 'balance': 1, 'account': 1, 'havent': 1, 'charged': 1, 'a': 1, 'debit': 1, 'card': 1, 'fees': 1}
{'no': 1, 'need': 1, 'to': 1, 'maintain': 1, 'minimum': 1, 'balance': 1}
{'not': 1, 'convenient': 1, 'to': 1, 'use': 1}
{'atm': 1, 'and': 1, 'branches': 1, 'are': 1, 'near': 1}
{'charge': 1, 'open': 1, 'a': 1, 'saving': 1, 'account': 1, 'all': 1, 'the': 1, 'process': 1, 'was': 1, 'done': 1, 'through': 1, 

In [107]:
df['tokens'] = df['review'].apply(lambda x: re.findall(r'[.?]|\w+', x))

In [108]:
def map_num(row):
	curr_dict = row['convenient'].copy()
	curr_dict.update(row['speed'])
	curr_dict.update(row['informative'])
	return [0 if curr_dict.get(i) is None else curr_dict.get(i) for i in row['tokens']]


In [109]:
df['ner_tags'] = df.apply(map_num, axis=1)

In [110]:
label_list = ['O', 'CON', "SPD", "INF"]
random_seed = 1
transformers.set_seed(seed=random_seed)

In [116]:
train_val, test = train_test_split(df, test_size=0.2, random_state=random_seed)

train_val = datasets.Dataset.from_pandas(train_val)
test = datasets.Dataset.from_pandas(test)

train_val = train_val.train_test_split(test_size=0.1, shuffle=True)
train, val = train_val['train'], train_val['test']

_nlp_data = {
	'train' : train,
	'validation' : val,
	'test' : test
}

nlp_data = datasets.DatasetDict(_nlp_data)

In [117]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [118]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.

    label_all_tokens (bool): A flag to indicate whether all tokens should have labels.
                             If False, only the first token of a word will have a label,
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [119]:
tokenized_datasets = nlp_data.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 200/200 [00:00<00:00, 647.51 examples/s]


In [122]:
metric = evaluate.load("seqeval")

In [123]:
def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }

In [124]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=7)

args = TrainingArguments(
	"test-ner",
	evaluation_strategy = "epoch",
	learning_rate=2e-5,
	per_device_train_batch_size=16,
	per_device_eval_batch_size=16,
	num_train_epochs=30,
	weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(tokenizer)

# I changed this part to the evaluate function
metric = evaluate.load("seqeval")

trainer = Trainer(
    model,
	args,
	train_dataset=tokenized_datasets["train"],
	eval_dataset=tokenized_datasets["validation"],
	data_collator=data_collator,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\anaconda3\envs\torchdev\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Yang LiTing\AppData\Local\Temp\ipykernel_154964\1029477858.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [125]:
trainer.train()

  3%|▎         | 45/1350 [07:34<3:13:37,  8.90s/it]c:\anaconda3\envs\torchdev\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: CON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\anaconda3\envs\torchdev\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: INF seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\anaconda3\envs\torchdev\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: SPD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\anaconda3\envs\torchdev\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                   
  3%|▎         | 45/1350 [07:52

{'eval_loss': 0.5617961883544922, 'eval_precision': 0.15422885572139303, 'eval_recall': 0.07294117647058823, 'eval_f1': 0.09904153354632587, 'eval_accuracy': 0.798686162322526, 'eval_runtime': 17.2813, 'eval_samples_per_second': 4.629, 'eval_steps_per_second': 0.289, 'epoch': 1.0}


                                                   
  7%|▋         | 90/1350 [15:41<4:07:52, 11.80s/it]

{'eval_loss': 0.49242669343948364, 'eval_precision': 0.2833333333333333, 'eval_recall': 0.12, 'eval_f1': 0.16859504132231404, 'eval_accuracy': 0.8328035600762873, 'eval_runtime': 19.9901, 'eval_samples_per_second': 4.002, 'eval_steps_per_second': 0.25, 'epoch': 2.0}


                                                          
 10%|█         | 135/1350 [1:27:21<2:15:50,  6.71s/it]

...

                                                     
 97%|█████████▋| 1305/1350 [5:03:00<09:24, 12.54s/it]

{'eval_loss': 0.8164464831352234, 'eval_precision': 0.38461538461538464, 'eval_recall': 0.3411764705882353, 'eval_f1': 0.3615960099750623, 'eval_accuracy': 0.8465776647594829, 'eval_runtime': 24.952, 'eval_samples_per_second': 3.206, 'eval_steps_per_second': 0.2, 'epoch': 29.0}


100%|██████████| 1350/1350 [5:10:28<00:00, 10.08s/it]c:\anaconda3\envs\torchdev\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: CON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\anaconda3\envs\torchdev\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: INF seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\anaconda3\envs\torchdev\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: SPD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
                                                     
100%|██████████| 1350/1350 [5:10:49<00:00, 13.81s/it]


{'eval_loss': 0.8171316981315613, 'eval_precision': 0.3891891891891892, 'eval_recall': 0.3388235294117647, 'eval_f1': 0.36226415094339615, 'eval_accuracy': 0.8480610298792117, 'eval_runtime': 18.2326, 'eval_samples_per_second': 4.388, 'eval_steps_per_second': 0.274, 'epoch': 30.0}
{'train_runtime': 18649.6199, 'train_samples_per_second': 1.158, 'train_steps_per_second': 0.072, 'train_loss': 0.16152079900105795, 'epoch': 30.0}


TrainOutput(global_step=1350, training_loss=0.16152079900105795, metrics={'train_runtime': 18649.6199, 'train_samples_per_second': 1.158, 'train_steps_per_second': 0.072, 'total_flos': 950673236050560.0, 'train_loss': 0.16152079900105795, 'epoch': 30.0})

In [131]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\vocab.txt',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [132]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id

In [128]:
json.dump(config, open("ner_model/config.json","w"))

In [133]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

In [139]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

example = "ATM at Bank XYZ was super convenient and very quick."

ner_results = nlp(example)

for i in ner_results:
	print(i)

{'entity': 'LABEL_0', 'score': 0.79301125, 'index': 1, 'word': 'atm', 'start': 0, 'end': 3}
{'entity': 'LABEL_0', 'score': 0.8505211, 'index': 2, 'word': 'at', 'start': 4, 'end': 6}
{'entity': 'LABEL_0', 'score': 0.98813397, 'index': 3, 'word': 'bank', 'start': 7, 'end': 11}
{'entity': 'LABEL_0', 'score': 0.9679319, 'index': 4, 'word': 'x', 'start': 12, 'end': 13}
{'entity': 'LABEL_0', 'score': 0.9699092, 'index': 5, 'word': '##y', 'start': 13, 'end': 14}
{'entity': 'LABEL_0', 'score': 0.97378594, 'index': 6, 'word': '##z', 'start': 14, 'end': 15}
{'entity': 'LABEL_1', 'score': 0.7261997, 'index': 7, 'word': 'was', 'start': 16, 'end': 19}
{'entity': 'LABEL_1', 'score': 0.92993605, 'index': 8, 'word': 'super', 'start': 20, 'end': 25}
{'entity': 'LABEL_1', 'score': 0.99079925, 'index': 9, 'word': 'convenient', 'start': 26, 'end': 36}
{'entity': 'LABEL_0', 'score': 0.72142243, 'index': 10, 'word': 'and', 'start': 37, 'end': 40}
{'entity': 'LABEL_2', 'score': 0.8704056, 'index': 11, 'word'